In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import random
from PIL import Image
import cv2 as cv
import tensorflow as tf 
import tensorflow_hub as hub 
import shutil 
#shutil.copy(src="../input/tokenization/tokenization.py",dst="./")
#! cp  -r "../input/bert-en-uncased-l-12-h-768-a-12-1" "./"
#import tokenization
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import math
from tensorflow.keras.utils import Sequence 
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

# Load datas:

In [ ]:
test = pd.read_csv("../input/shopee-product-matching/test.csv")
images_path = "../input/shopee-product-matching/test_images"
if len(test) <= 3 :
    test = pd.read_csv("../input/shopee-product-matching/train.csv")
    images_path = "../input/shopee-product-matching/train_images"

# Useful functions :

In [ ]:
SEED = 42

In [ ]:
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.000005 * BATCH_SIZE
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
    return lr_callback


def processing_data(df):
    lb = LabelEncoder()
    df["encoded_label_group"] = lb.fit_transform(df["label_group"])
    N_CLASSES = df["encoded_label_group"].nunique()
    #xtr,xts,ytr,yts = train_test_split(df["cleaned_title"].values,df["encoded_label_group"].\
                                       #values,stratify=df["encoded_label_group"].values,\
                                       #test_size =0.33,random_state=SEED)
    return N_CLASSES



In [ ]:
class DataGenerator(Sequence):
    def __init__(self,df,img_size = 256,path=images_path,listID=None,\
                batch_size =32,shuffle=True) :
        self.df = df 
        self.path = path 
        self.img_size = img_size
        self.batch_size = batch_size
        self.listID = listID
        #self.labels = labels
        self.shuffle =shuffle
        self.on_epoch_end()
        #self.indexes = np.arange(len(self.df))
    def __len__(self):
        cls = len(self.listID) // self.batch_size
        cls += int((len(self.listID) % self.batch_size) !=0)
        return cls
    def __getitem__(self,index):
        indices = self.indexes[index * self.batch_size:(index+1) * self.batch_size]
        list_ID_temp = [self.listID[k] for k in indices]
        X,Y = self.__data_generation(list_ID_temp)
        X = X,Y
        return X,Y
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.listID))
        if self.shuffle :
            np.random.shuffle(self.indexes)
    def __data_generation(self,list_ID_temp):
        images = np.zeros((len(list_ID_temp),self.img_size,self.img_size,3),dtype=np.float)
        dff = self.df.loc[list_ID_temp]
        y = np.zeros((len(list_ID_temp)),dtype=int)
        for i,(j,row) in enumerate(dff.iterrows()):
            img = os.path.join(self.path,row.image)
            img = cv.imread(img)
            img = cv.resize(img,(self.img_size,self.img_size))
            images[i,] = img
            y[i] = row.encoded_label_group
        return images,y

In [ ]:
def effnet():
     margin = ArcMarginProduct(
            n_classes = N_CLASSES, 
            s = 30, 
            m = 0.5, 
            name='head/arc_margin', 
            dtype='float32'
            )
     inp = tf.keras.layers.Input(shape=(256,256,3),dtype=tf.float32)
     y= tf.keras.Input(shape=(),dtype = tf.float32 ,name = "label" )
     effnet = EfficientNetB0(weights=AWG,include_top = False ,pooling="AVG",input_shape=None)
     #inp = tf.keras.layers.BatchNormalization()(inp)
     #inp = tf.keras.layers.Dropout(0.3)(inp)
     out = effnet(inp)
     #out = tf.keras.layers.BatchNormalization()(out)
     #out = tf.keras.layers.Dropout(0.3)(out)
     out = tf.keras.layers.GlobalAvgPool2D()(out)
     x = margin([out,y])
     output = tf.keras.layers.Softmax()(x)
     model = tf.keras.models.Model(inputs = [inp,y],outputs=[output])
     model.compile(optimizer = tf.keras.optimizers.Adam(1e-5),loss=[tf.keras.losses.\
                   sparse_categorical_crossentropy],metrics="accuracy")
     return model 
def effnetmodelTrainAndSave(df):
    seed_everything(SEED)
    md = effnet()
    #indices = list(df.index)
    #train_indices = np.random.choice(indices,int(0.8 * len(indices)),replace=False)
    #test_indices = np.array([i for i in indices if i not in train_indices])
    #st = StratifiedKFold(2,shuffle=False)
    #for train_index,test_index in st.split(df.values,df["encoded_label_group"].values):
    train_index,test_index,ytr,yts = train_test_split(df.index,df["encoded_label_group"].values,test_size=0.35,\
                                      stratify=df["encoded_label_group"].values)
    train_dataset = DataGenerator(df,listID = train_index)
    val_dataset = DataGenerator(df,listID = test_index)
    
    checkpoints = tf.keras.callbacks.ModelCheckpoint(f"effnet_weights.h5",\
                                                             monitor = "val_loss",\
                                                             verbose = VERBOSE,
                                                             save_best_only = True,\
                                                             save_weights_only = True,\
                                                             mode = "min")
  
    history = md.fit(train_dataset,validation_data=val_dataset,callbacks=[checkpoints,get_lr_callback()],\
                     use_multiprocessing= True,workers =6,epochs=EPOCHS)

# Data preparation :

In [ ]:
label_group = test.groupby("label_group").posting_id.unique()
test["target"] = test["label_group"].map(label_group)

In [ ]:
N_CLASSES= processing_data(test)

In [ ]:
print(N_CLASSES)

# Modeling :

In [ ]:
VERBOSE = 1
EPOCHS = 30
BATCH_SIZE = 32
SEED = 42
AWG = "../input/effnetb0/efficientnetb0_notop.h5"

In [ ]:
effnetmodelTrainAndSave(test)